# Heat Isocalor Calculation for POIs

In [1]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import openrouteservice as ors
from pathlib import Path
import pyogrio
import os
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from pyproj import Geod
import rasterio
import rasterio.warp
import numpy as np
from rasterstats import zonal_stats

In [ ]:
# Change working directory to parent folder
os.chdir("..")  # Move up one directory level

# Check working directory
print(f"Working directory: {os.getcwd()}")

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Prepare the OpenRouteService client for HEAL-API requests
ors_url = "https://heal.openrouteservice.org/api-iso/ors/"

profile = "foot-walking"
fmt = 'geojson'
__headers = {"headers":{
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Content-Type': 'application/json; charset=utf-8'}}

client = ors.Client(base_url=ors_url)

In [5]:
# Define input parameters for Transport POIs
input_filename = 'pois_hd_transport_osm.geojson'
pois_name = 'transport'

# Define file path
filepath = f'{Path.cwd()}\data\\{input_filename}'

# Read the shapefile
points = gpd.read_file(filepath)   

# Access the geometry column to get the points
point_geometry = points.geometry

# Seperate the coordinates into a list
coordinates = [[point.x, point.y] for point in point_geometry]

# Print the number of points
print(f'Number of points: {len(coordinates)}')

Number of points: 356


In [6]:
# Define a subset of the coordinates (for testing purposes only)
# coordinates = coordinates[:10]
# coordinates

[[8.7561098, 49.4160568],
 [8.6628297, 49.4056451],
 [8.6723434, 49.3830967],
 [8.67217, 49.3841071],
 [8.6589861, 49.3786647],
 [8.6912464, 49.3999244],
 [8.7108, 49.4136759],
 [8.7086199, 49.4133846],
 [8.708932, 49.4152724],
 [8.7311542, 49.3881645]]

Define Function `get_isocalors` that returns the isochrones of a heat equation from the HEAL-API.

In [7]:
# define function to call isochrones from HEAL-API

def get_isocalors(coordinates, range_values, range_type, shadow_factor, timeofday):
  """
  Function to call isochrones from HEAL-API.

  Args:
    coordinates (str): The coordinates of the location.
    range_values (list): The range values for the isochrones.
    range_type (str): The type of range values (e.g., 'time', 'distance').
    shadow_factor (float): The shadow factor for the isochrones.
    timeofday (str): The time of day for the isochrones (e.g., 'morning', 'noon', 'afternoon', 'evening').

  Returns:
    dict: The response from the HEAL-API.

  """
  # Define the parameters for the API request
  params = {
    'locations': coordinates,
    'range': range_values,
    'range_type': range_type,
    "options": {
      "profile_params": {
        "weightings": {
          "csv_factor": shadow_factor,
          "csv_column": timeofday
        }
      }
    }
  }

  # Make the API request
  response = client.request(
    url="v2/isochrones/{}/{}".format(profile, fmt),
    post_json=params,
    requests_kwargs=__headers,
    get_params=[]
  )

  return response

Call function for various parameter sets

In [8]:
# Run the function for all coordinates and different time of day values and different shadow factors:
time_of_day_values = ['morning','noon', 'afternoon', 'evening'] # can be: morning, noon, afternoon, evening
shadow_factors = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0] # range from 0.0 to 5.0 (0.0 = no influence of shadow-data, 1.0 - 5.0 = increasing heat sensibility)
range_vals = [[300],[600],[900],[1200],[1800]] # in seconds

# Create an empty GeoDataFrame to store the isochrones
isocalors_df = gpd.GeoDataFrame()

# Iterate over range values
for range_val in range_vals:
    # Iterate over the time of day values
    for timeod in time_of_day_values:
        # Iterate over the shadow factors
        for shadow_fact in shadow_factors:
            # Iterate over all POIs
            for c in coordinates:
                isochrones = get_isocalors([list(c)], range_val, 'time', shadow_fact, timeod)
                isocalors_df_temp = gpd.GeoDataFrame.from_features(isochrones, crs=points.crs)
                
                # Access for c corresponding osm_id, fclass, and name
                point_index = coordinates.index(c)
                osm_id = points.iloc[point_index]['osm_id'] if 'osm_id' in points.columns else np.nan
                fclass = points.iloc[point_index]['fclass'] if 'fclass' in points.columns else np.nan
                name = points.iloc[point_index]['name']
                
                # Add shadow factor and time of day, osm_id, fclass and name as columns
                isocalors_df_temp['shadow_factor'] = shadow_fact
                isocalors_df_temp['timeofday'] = timeod
                isocalors_df_temp['osm_id'] = osm_id
                isocalors_df_temp['fclass'] = fclass
                isocalors_df_temp['name'] = name
                isocalors_df_temp['start_point'] = [c]
                
                # Append the temporary GeoDataFrame to the main GeoDataFrame
                isocalors_df = isocalors_df.append(isocalors_df_temp, ignore_index=True)

Calculate the population within the isocalors with Zensus 2022 data

In [9]:
# Specify the path to the raster file
raster_file = f'{Path.cwd()}\\data\\zensus_2022_hd_LAEA.tif'

# Read the population raster file
with rasterio.open(raster_file) as dataset:
    # Get the population data
    population_data = dataset.read(1)

# Set geometry row
isocalors_df.set_geometry('geometry', inplace=True)

# Reproject the isochrones to same CRS as the population data
isocalors_df = isocalors_df.to_crs(dataset.crs)

# Initialize an empty list to store the population for each isochrone
population_values = []

# Iterate over the rows of the GeoDataFrame
for index, row in isocalors_df.iterrows():
    # Get the geometry of the current row
    geometry = row['geometry']
    
    # Compute the population for the current isochrone
    population = zonal_stats(geometry, population_data, stats="sum", nodata=-200, affine=dataset.transform, all_touched=False)[0]['sum'] # all_touched=False does not include the population of the surrounding pixels
    
    # Append the population to the list
    population_values.append(population)

# Add the population values to the GeoDataFrame
isocalors_df['population'] = population_values
isocalors_df = isocalors_df.to_crs(epsg=25832)

Calculate the number of transport stops in the isocalors

In [10]:
# Load transport POIs
filepath = f'{Path.cwd()}\data\\pois_hd_transport_osm.geojson'

# Read the GeoJSON file
pois = gpd.read_file(filepath)
pois = pois.to_crs(epsg=25832)

# Filter duplicate POIs based on the 'name' column
pois = pois.drop_duplicates(subset='name')

# Perform spatial join
joined = gpd.sjoin(pois, isocalors_df, how='inner', predicate='within')

# Count POIs in each polygon
poi_counts = joined.groupby(joined.index_right).size()

# Add the 'pois_num' column to the original polygons GeoDataFrame
isocalors_df['pois_num'] = poi_counts

# Fill NaN values with 0 for polygons with no POIs
isocalors_df['pois_num'] = isocalors_df['pois_num'].fillna(0)

# Calculate the POI density per square kilometer
isocalors_df['pois_density_km2'] = isocalors_df['pois_num'] / (isocalors_df.area / 10**6)

Identify City Quarter of every isochrone for later analysis

In [11]:
# get city quarter boundaries
filepath = f'{Path.cwd()}\data\\hd_quarter_boundaries.shp' # shapefile with city quarter boundaries (admin_level = 9) downloaded for Heidelberg from https://osm-boundaries.com/ and available in the data folder

# Read the shapefile
quarters = gpd.read_file(filepath)

# Reproject the GeoDataFrame to EPSG: 4326
isocalors_df = isocalors_df.to_crs(epsg=4326)
quarters = quarters.to_crs(isocalors_df.crs)

# Transform Start Points to Point Geometry
isocalors_df['start_point'] = [Point(c) for c in isocalors_df['start_point']]

# Set geometry row
isocalors_df.set_geometry('start_point', inplace=True)

# Create the 'quarter' column
isocalors_df['quarter'] = None

# Loop over the start points and write corresponding quarter to new column
for index, row in isocalors_df.iterrows():
    for quarter_index, quarter_row in quarters.iterrows():
        if row['start_point'].intersects(quarter_row['geometry']):
            isocalors_df.loc[index, 'quarter'] = quarter_row['name']

# Set geometry row
isocalors_df.set_geometry('geometry', inplace=True)

# Transform Start Point Geomentries back to Coordinate Pairs for export
isocalors_df['start_point'] = [[point.x, point.y] for point in isocalors_df['start_point']]

# Show the first 5 rows of the GeoDataFrame
print(isocalors_df.head())

# Write the GeoDataFrame to a GeoJSON file
pyogrio.write_dataframe(isocalors_df, f'{Path.cwd()}\data\\isocalor_{pois_name}_all.json')

                                            geometry  group_index  value  \
0  POLYGON ((8.75006 49.41520, 8.75012 49.41509, ...            0  300.0   
1  POLYGON ((8.65706 49.40630, 8.65899 49.40426, ...            0  300.0   
2  POLYGON ((8.66712 49.38260, 8.66799 49.38116, ...            0  300.0   
3  POLYGON ((8.66643 49.38410, 8.66717 49.38339, ...            0  300.0   
4  POLYGON ((8.65586 49.38098, 8.65743 49.37628, ...            0  300.0   

                                    center  shadow_factor timeofday  \
0  [8.756109344710678, 49.416061012027534]            0.0   morning   
1  [8.662839110211735, 49.405671770459755]            0.0   morning   
2   [8.672425625389886, 49.38309314274967]            0.0   morning   
3   [8.672161088448648, 49.38406133449652]            0.0   morning   
4   [8.659095376747402, 49.37868147724087]            0.0   morning   

            osm_id  fclass                 name              start_point  \
0   node/300751575     NaN      Adler-Üb

Calculate affected and non-affected areas with population data

In [12]:
# Calculate the areas of isochrones for every combination of timeofday, shadow_factor, and value

# Read in the isocalor file to reduce computation time (if the file is already created)
# isocalors_file = f'{Path.cwd().parent}\\heal\\data\\isocalor_transport_heal_5_10_15_20_30_all.json' # for test purposes only
isocalors_file = f'{Path.cwd()}\data\\isocalor_{pois_name}_all.json'
isocalors_df = gpd.read_file(isocalors_file)

# Create an empty GeoDataFrame to store the unitary union objects
unitary_union_df = gpd.GeoDataFrame()

# Get unique combinations of timeofday, shadow_factor, and value
combinations = isocalors_df[['timeofday', 'shadow_factor', 'value']].drop_duplicates()

# Iterate over the combinations
for index, row in combinations.iterrows():
    # Filter the isochrones dataframe for the current combination
    filtered_df = isocalors_df[(isocalors_df['timeofday'] == row['timeofday']) &
                               (isocalors_df['shadow_factor'] == row['shadow_factor']) &
                               (isocalors_df['value'] == row['value'])]
    
    # Create the unitary union object
    unitary_union = filtered_df['geometry'].unary_union
    
    # Create a new row for the unitary union object in the unitary_union_df
    unitary_union_df = unitary_union_df.append({'timeofday': row['timeofday'],
                                                'shadow_factor': row['shadow_factor'],
                                                'value': row['value'],
                                                'geometry': unitary_union}, ignore_index=True)

# Set the geometry column of the unitary_union_df to the unitary union objects
unitary_union_df = unitary_union_df.set_geometry('geometry', crs=isocalors_df.crs)

In [13]:
# Get the vulnerable areas by calculating the difference between the city boundary and the unitary union objects

# Read the city boundaries
filepath = f'{Path.cwd()}\data\\hd_boundaries.shp' # shapefile with city boundaries (admin_level = 6) downloaded for Heidelberg from https://osm-boundaries.com/ and available in the data folder
hd_boundary = gpd.read_file(filepath)

# Reproject the city boundary to the same CRS as the unitary union objects
hd_boundary = hd_boundary.to_crs(unitary_union_df.crs)

# Extract the geometry of the single object
hd_boundary_geom = hd_boundary.geometry.iloc[0]

# Function to calculate the difference
def calculate_difference(geometry):
    return hd_boundary_geom.difference(geometry)

# Apply the difference calculation to each geometry in the GeoDataFrame
differences = unitary_union_df.geometry.apply(calculate_difference)

# Create a new GeoDataFrame with the difference geometries
difference_gdf = gpd.GeoDataFrame(geometry=differences, crs=unitary_union_df.crs)

# Add an index column to relate differences back to original objects
difference_gdf['original_index'] = unitary_union_df.index

# Add the timeofday, shadow_factor, and value columns to the difference GeoDataFrame
difference_gdf = difference_gdf.join(unitary_union_df.drop(columns='geometry'))

# Cut the unitary union objects with the city boundary
unitary_union_df = gpd.overlay(unitary_union_df, hd_boundary, how='intersection')
unitary_union_df = unitary_union_df[['timeofday', 'shadow_factor', 'value', 'geometry']]

In [14]:
# Calculate population for each reachable object with Zensus 2022 data

# Specify the path to the raster file
raster_file = f'{Path.cwd()}\\data\\zensus_2022_hd_LAEA.tif'

# Read the population raster file
with rasterio.open(raster_file) as dataset:
    # Get the population data
    population_data = dataset.read(1)

# Reproject the areas to same CRS as the population data
unitary_union_df = unitary_union_df.to_crs(dataset.crs)

# Initialize an empty list to store the population for each isochrone
population_values = []

# Iterate over the rows of the GeoDataFrame
for index, row in unitary_union_df.iterrows():
    # Get the geometry of the current row
    geometry = row['geometry']
    
    # Compute the population for the current isochrone
    population = zonal_stats(geometry, population_data, stats="sum", nodata=-200, affine=dataset.transform, all_touched=False)[0]['sum']

    # Append the population to the list
    population_values.append(population)

# Add the population values to the GeoDataFrame
unitary_union_df['population'] = population_values
unitary_union_df = unitary_union_df.to_crs(epsg=25832)

# Export the unitary union objects to a GeoJSON file
pyogrio.write_dataframe(unitary_union_df, f'{Path.cwd()}\data\\vul_area_masks_{pois_name}_all.json')

In [15]:
# Calculate population for each non-reachable object with Zensus 2022 data
# Reproject the areas to same CRS as the population data
difference_gdf = difference_gdf.to_crs(dataset.crs)

# Initialize an empty list to store the population for each isochrone
population_values = []

# Iterate over the rows of the GeoDataFrame
for index, row in difference_gdf.iterrows():
    # Get the geometry of the current row
    geometry = row['geometry']
    
    # Compute the population for the current isochrone
    population = zonal_stats(geometry, population_data, stats="sum", affine=dataset.transform, all_touched=False)[0]['sum']

    # Append the population to the list
    population_values.append(population)


# Add the population values to the GeoDataFrame
difference_gdf['population'] = population_values
difference_gdf = difference_gdf.to_crs(epsg=25832)

# Export the difference objects objects to a GeoJSON file
pyogrio.write_dataframe(difference_gdf, f'{Path.cwd()}\data\\vul_areas_{pois_name}_all.json')

# Analysis of POIs numbers in Transport_Isocalors

In [16]:
def process_pois(pois, isocals):
    if pois.crs != isocals.crs:
        pois = pois.to_crs(isocals.crs)
    filtered_pois = gpd.sjoin(pois, isocals, predicate='within').drop(columns=['index_right'])
    return len(pois), len(filtered_pois)

# Define file paths and read isocalors
isocals = gpd.read_file(f'{Path.cwd()}\data\\vul_area_masks_{pois_name}_all.json')
isocals = isocals[(isocals['timeofday']=='noon') & (isocals['shadow_factor']==5.0) & (isocals['value']==900.0)] # Filter for specific scenario if necessary

results = []

# Process OSM POIs
osm_pois = gpd.read_file(f'{Path.cwd()}\data\\pois_hd_osm.geojson')
for category in ['clinic', 'doctors', 'hospital', 'pharmacy', 'supermarket', 'senior_facility', 'kindergarten']:
    pois_temp = osm_pois[osm_pois['category'] == category]
    total, filtered = process_pois(pois_temp, isocals)
    results.append({'pois_category': category, 'total_points': total, 'filtered_points': filtered})

# Create DataFrame and calculate percentages
results_df = pd.DataFrame(results)
results_df['filtered%'] = (results_df['filtered_points'] / results_df['total_points']) * 100

# Calculate totals and add a final row
total_points = results_df['total_points'].sum()
total_filtered_points = results_df['filtered_points'].sum()
total_percentage = (total_filtered_points / total_points) * 100
total_row = pd.DataFrame([{'pois_category': 'Total','total_points': total_points, 'filtered_points': total_filtered_points, 'filtered%': total_percentage}])
results_df = pd.concat([results_df, total_row], ignore_index=True)

# Export results
results_df.to_csv(f'{Path.cwd()}\data\\POIs_difference_noon_5_15.csv', index=False)

In [17]:
# Get the filtered POIs for the selected isochrones

def filter_pois(pois, isocals, pois_name):
    if pois.crs != isocals.crs:
        pois = pois.to_crs(isocals.crs)
    filtered_pois = gpd.sjoin(pois, isocals, predicate='within').drop(columns=['index_right'])
    filtered_pois['category'] = pois_name
    return filtered_pois

results_list = []

# Process OSM POIs
osm_pois = gpd.read_file(f'{Path.cwd()}\data\\pois_hd_osm.geojson')
for category in ['clinic', 'doctors', 'hospital', 'pharmacy', 'supermarket', 'senior_facility', 'kindergarten']:
    pois_temp = osm_pois[osm_pois['category'] == category]
    pois_name = category
    filtered = filter_pois(pois_temp, isocals, pois_name)
    results_list.append(filtered)

# Create GeoDataFrame from results
results_gdf = gpd.GeoDataFrame(pd.concat(results_list, ignore_index=True), crs=isocals.crs)
results_gdf = results_gdf.set_geometry('geometry')

pois_name = 'transport'

# Export results
pyogrio.write_dataframe(results_gdf, f'{Path.cwd()}\data\\filtered_pois_noon_5_15.json')

Calculate the ratio between area of isocalor of heat factor 0 and area of isocalors of heat factors > 0

In [18]:
# Define funtion to calculate the area difference by osm_id
def calculate_area_difference_by_osm_id(geodataframe):

    ''' Function to calculate the area difference for each object with the same osm_id in a geodataframe.The function groups 
    the geodataframe by osm_id and calculates the area difference for each object in the group. The function appends the 
    area difference to the geodataframe for each object with the same osm_id.'''
     
    # Group the geodataframe by osm_id
    grouped = geodataframe.groupby('osm_id')
    
    # Create an empty list to store the area differences and percentage values
    area_differences = []
    area_differences_perc = []
    
    # Iterate over each group
    for osm_id, group in grouped:
        # Calculate the area difference for each object in the group
        area_diff = group['geometry'].area.max() - group['geometry'].area.min()
        # Calculate the percentage difference
        area_diff_perc = area_diff / group['geometry'].area.max() * 100
        
        # Append the area difference to the list
        area_differences.append(area_diff)
        area_differences_perc.append(area_diff_perc)


        # Append the area difference to the geodataframe for each object with the same osm_id
        geodataframe.loc[geodataframe['osm_id'] == osm_id, ['area_difference', 'area_diff_perc']] = [area_diff, area_diff_perc]
    
    return geodataframe

In [19]:
isocalors_df

,group_index,value,shadow_factor,timeofday,osm_id,fclass,name,population,pois_num,pois_density_km2,quarter,geometry
0,0,300.0,0.0,morning,node/300751575,NaN,Adler-Überfahrt,140.0,2.0,23.393852,Schlierbach,"POLYGON ((8.75006 49.41520, 8.75012 49.41509, ..."
1,0,300.0,0.0,morning,node/4801418695,NaN,Agnesistraße,2192.0,3.0,11.589035,Bahnstadt,"POLYGON ((8.65706 49.40630, 8.65899 49.40426, ..."
2,0,300.0,0.0,morning,node/366396868,NaN,Albert-Fritz-Straße,2742.0,2.0,5.498026,Kirchheim,"POLYGON ((8.66712 49.38260, 8.66799 49.38116, ..."
3,0,300.0,0.0,morning,node/9864445805,NaN,Albert-Fritz-Straße,2600.0,2.0,5.445110,Kirchheim,"POLYGON ((8.66643 49.38410, 8.66717 49.38339, ..."
4,0,300.0,0.0,morning,node/8388878952,NaN,Alfred-Jost-Straße,2693.0,3.0,8.959942,Kirchheim,"POLYGON ((8.65586 49.38098, 8.65743 49.37628, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0,1800.0,5.0,evening,node/303222540,NaN,Alois-Link-Platz,7177.0,9.0,11.794924,Weststadt,"POLYGON ((8.68393 49.40167, 8.68424 49.40019, ..."
1196,0,1800.0,5.0,evening,node/300751566,NaN,Alte Brücke,9071.0,19.0,11.400153,Altstadt,"POLYGON ((8.69730 49.41023, 8.70094 49.40760, ..."
1197,0,1800.0,5.0,evening,way/321018184,NaN,Alte Brücke,10482.0,21.0,10.505779,Altstadt,"POLYGON ((8.69384 49.40987, 8.69550 49.40832, ..."
1198,0,1800.0,5.0,evening,node/297489765,NaN,Alte Brücke Nord,6101.0,12.0,10.078612,Neuenheim,"POLYGON ((8.69387 49.41322, 8.69427 49.41312, ..."


In [20]:
# Call the function to calculate the area difference for the isocalors
# Get unique range values
range_vals = isocalors_df['value'].unique()

# Create an empty GeoDataFrame to store the results
isocalors_df_area = gpd.GeoDataFrame()

# Get unique time of day values

time_of_day_values = isocalors_df['timeofday'].unique()

# Get unique shadow factors
shadow_facts = isocalors_df['shadow_factor'].unique()[1:]

# Iterate over time of day values to filter the GeoDataFrame
for time_of_day in time_of_day_values:
    # Slice the GeoDataFrame for the current time of day
    isocalors_df_tod = isocalors_df[isocalors_df['timeofday'] == time_of_day]

    # Iterate over shadow factors to filter the GeoDataFrame
    for shadow_fact in shadow_facts:
        # Slice the GeoDataFrame for the current shadow factor
        isocalors_df_temp = isocalors_df_tod[isocalors_df_tod['shadow_factor'].isin([0.0, shadow_fact])]
        
        # Iterate over the unique range values
        for range_val in range_vals:
            # Filter the GeoDataFrame to get the isocalors for the current range value
            isocalors_df_range = isocalors_df_temp[isocalors_df_temp['value'] == range_val]

            # Calculate the area difference for the isocalors
            isocalors_df_range = calculate_area_difference_by_osm_id(isocalors_df_range)

            # Append the GeoDataFrame to the main GeoDataFrame
            isocalors_df_area = pd.concat([isocalors_df_area, isocalors_df_range], ignore_index=True)

isocalors_df_area = isocalors_df_area[isocalors_df_area['shadow_factor'] > 0.0]  

# Export the isocalors with area differences to a GeoJSON file
pyogrio.write_dataframe(isocalors_df_area, f'{Path.cwd()}\data\\isocalor_area_transport_all.json')

Calculate the population difference between isocalore of heat factor 0 and isochrone of heat factors > 0

In [21]:
# Define funtion to calculate the population difference by osm_id
def calculate_pop_difference_by_osm_id(geodataframe):

    '''Function to calculate the population difference by osm_id. The function groups the geodataframe by osm_id and 
    calculates the population difference for each object in the group. The function appends the population difference 
    to the geodataframe for each object with the same osm_id.'''

    # Group the geodataframe by osm_id
    grouped = geodataframe.groupby('osm_id')
    
    # Create an empty list to store the population differences and percentage values
    pop_differences = []
    pop_differences_perc = []
    
    # Iterate over each group
    for osm_id, group in grouped:
        # Calculate the area difference for each object in the group
        pop_diff = group['population'].max() - group['population'].min()

        if group['population'].max() != 0:
            # Calculate the percentage difference
            pop_diff_perc = pop_diff / group['population'].max() * 100
        else:
            pop_diff_perc = 0
        
        # Append the area difference to the list
        pop_differences.append(pop_diff)
        pop_differences_perc.append(pop_diff_perc)


        # Append the area difference to the geodataframe for each object with the same osm_id
        geodataframe.loc[geodataframe['osm_id'] == osm_id, ['pop_difference', 'pop_diff_perc']] = [pop_diff, pop_diff_perc]
    
    return geodataframe

In [22]:
# Get unique range values
range_vals = isocalors_df['value'].unique()

# Create an empty GeoDataFrame to store the results
isocalors_df_pop = gpd.GeoDataFrame()

# Get unique shadow factors
shadow_facts = isocalors_df['shadow_factor'].unique()[1:]

# Get unique time of day values
time_of_day_values = isocalors_df['timeofday'].unique()

# Iterate over time of day values
for timeod in time_of_day_values:
    # Slice the GeoDataFrame for the current time of day
    isocalors_df_tod = isocalors_df[isocalors_df['timeofday'] == timeod]

    # Iterate over shadow factors to filter the GeoDataFrame
    for shadow_fact in shadow_facts:
        # Slice the GeoDataFrame for the current shadow factor
        isocalors_df_temp = isocalors_df_tod[isocalors_df_tod['shadow_factor'].isin([0.0, shadow_fact])]
        # isocalors_df_temp = isocalors_df[isocalors_df['shadow_factor'].isin([0.0, shadow_fact])]

        # Iterate over the unique range values
        for range_val in range_vals:
            # Filter the GeoDataFrame to get the isocalors for the current range value
            isocalors_df_range = isocalors_df_temp[isocalors_df_temp['value'] == range_val]

            # Calculate the area difference for the isocalors
            isocalors_df_range = calculate_pop_difference_by_osm_id(isocalors_df_range)

            # Append the GeoDataFrame to the main GeoDataFrame
            # isocalors_df_pop = isocalors_df_pop.append(isocalors_df_range, ignore_index=True)
            isocalors_df_pop = pd.concat([isocalors_df_pop, isocalors_df_range], ignore_index=True)


isocalors_df_pop = isocalors_df_pop[isocalors_df_pop['shadow_factor'] > 0.0]

# Export the isocalors with population differences to a GeoJSON file
pyogrio.write_dataframe(isocalors_df_pop, f'{Path.cwd()}\data\\isocalor_pop_transport_all.json')